In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 有效广告及其特征

In [3]:
import pandas as pd
from pathlib import Path
import os, re
from functools import reduce, partial
from utils import or_func

2019-05-12 19:17:45,086	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-12_19-17-45_3183/logs.
2019-05-12 19:17:45,191	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:12673 to respond...
2019-05-12 19:17:45,315	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:20631 to respond...
2019-05-12 19:17:45,320	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-12 19:17:45,344	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-12_19-17-45_3183/logs.
2019-05-12 19:17:45,348	WARNING services.py:1304 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 33736691712 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-

In [4]:
root = Path('../data')
root
os.listdir(root)

PosixPath('../data')

['2019-03-10_log_exposed.csv',
 '2019-02-23_log.csv',
 '2019-02-28_log.csv',
 '2019-03-14_log_exposed.csv',
 '2019-02-22_log_exposed.csv',
 'log_label.tgz',
 '2019-03-16_log.csv',
 '2019-03-11_log_exposed.csv',
 '2019-02-16_log_exposed.csv',
 '2019-03-12_log_exposed.csv',
 '2019-02-28_log_exposed.csv',
 '2019-03-06_log.csv',
 '2019-03-13_log.csv',
 '2019-02-23_log_exposed.csv',
 '2019-03-19_log.csv',
 '2019-03-01_log_exposed.csv',
 '2019-02-26_log_exposed.csv',
 '2019-03-01_log.csv',
 '2019-02-20_log_exposed.csv',
 '2019-02-19_log_exposed.csv',
 '2019-02-24_log.csv',
 '2019-03-03_log.csv',
 '2019-02-27_log_exposed.csv',
 '2019-03-03_log_exposed.csv',
 '2019-02-17_log.csv',
 '2019-02-18_log.csv',
 '2019-03-05_log_exposed.csv',
 '2019-03-07_log_exposed.csv',
 '2019-02-16_log.csv',
 '2019-02-26_log.csv',
 '.ipynb_checkpoints',
 '2019-03-09_log.csv',
 '2019-03-15_log.csv',
 '2019-02-21_log.csv',
 '2019-03-11_log.csv',
 'test_set.csv',
 'users_data.csv',
 '2019-03-07_log.csv',
 '2019-03-13_

In [5]:
ads = pd.read_csv(root/'ad_static_operation.csv', encoding='utf-8')
ads.head()
ads.info()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,2,100
1,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,3,"age:217,601,79,202,837,942,638,394,347,731,739..."
2,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,4,"281474976694272,281474976694272,28147497669427..."
3,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-22 23:37:02,1,1,0
4,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-23 00:13:15,1,2,100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614182 entries, 0 to 614181
Data columns (total 11 columns):
广告id       614182 non-null int64
创建时间       614182 non-null object
广告账户id     614182 non-null int64
商品id       614182 non-null int64
商品类型       614182 non-null int64
广告行业id     614182 non-null int64
素材尺寸       614182 non-null int64
创建/修改时间    614182 non-null object
操作类型       614182 non-null int64
修改字段       614182 non-null int64
操作后的字段值    614182 non-null object
dtypes: int64(8), object(3)
memory usage: 51.5+ MB


- 操作后的字段值:
    - 广告状态取值:1- 正常,0-失效
    - 出价:整数(单位分)
    - 投放时段:字符串
    - 人群定向:字符串。格式如下: feature_name1:feature_value1,feature_value2|feature_name2:feature_value3,feature_value4|...
        此处 feature_name 取值同用户属性文件中的各列属性名,feature_value 取值 id 同用户属性文件中的定义,不同feature 用“|”分隔,不同 feature 取值用逗号分隔。
      广告通过人群定向
的设置来召回对应的用户请求,对应的人群规则:不同 feature_name 是
求 交 集 , 同 一 featurename 下 不 同 的 value 求 并 集 , 未 定 义 的
feature_name 则 表 示 该 维 度 不 限 。 举 例 如 : 定 向 设 置 为
age:51,62,73,84|gender:1|area:1,3,5 ; 则表示该广告能被 “
(年龄 id 为
51 或 62 或 73 或 84) 且 (性别取值为 1) 且 (地域取值为 1 或 3
或 5) ”的用户召回(即在这些用户上有曝光机会)

In [6]:
users = pd.read_csv(root/'users_data.csv', encoding='utf-8')
users.head()
users.info()

,用户id,Age,Gender,地域,婚恋状态,学历,消费能力,设备,工作状态,连接类型,行为兴趣
0,624218,753,2,"523,7589,12527,9019,14605,13489,302,6222,14214...",0,6,1,2,0,4,"32612,34406,3413,33059,4275,4667,4561,34051,42..."
1,271475,601,3,"9019,9032,663,12527,14532,2416,2599,1987,9687,...",13,7,2,2,0,2,"4990,4992,3413,33061,4115,4876,2393,34167,3441..."
2,44439,819,3,"12527,14188,9841,9019,14605,6222,9962,6643,172...",13,7,3,2,4,4,"3099,4403,34414,431,34415,5108,31686,3240,3229..."
3,863717,1,2,"12527,3435,2398,11069,1292,9019,14605,14348,62...",6,5,3,2,0,2,"34293,845,3413,3099,33061,33059,4120,4410,4415..."
4,1305469,714,3,"8928,9019,1540,719,12527,6051,10545,12118,7107...",13,5,3,2,0,2,"32612,34289,34406,3413,4275,4410,4561,34051,34..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396718 entries, 0 to 1396717
Data columns (total 11 columns):
用户id      1396718 non-null int64
Age       1396718 non-null int64
Gender    1396718 non-null int64
地域        1396718 non-null object
婚恋状态      1396718 non-null object
学历        1396718 non-null int64
消费能力      1396718 non-null int64
设备        1396718 non-null int64
工作状态      1396718 non-null object
连接类型      1396718 non-null int64
行为兴趣      1396718 non-null object
dtypes: int64(7), object(4)
memory usage: 117.2+ MB


### 多值特征的处理
- 离散为 bool 特征

In [8]:
users.shape

(1396718, 11)

## 广告操作数据

In [9]:
ads.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
614177,412811,2019-02-21 21:39:18,17804,26944,13,12,34,2019-02-21 21:39:18,2,2,200
614178,412811,2019-02-21 21:39:18,17804,26944,13,12,34,2019-02-21 21:39:18,2,3,"age:217,601,79,202,837,942,638,394,347,731,739..."
614179,412811,2019-02-21 21:39:18,17804,26944,13,12,34,2019-02-21 21:39:18,2,4,"281474976694272,281474976694272,28147497669427..."
614180,412811,2019-02-21 21:39:18,17804,26944,13,12,34,2019-02-23 05:40:01,1,2,131
614181,412811,2019-02-21 21:39:18,17804,26944,13,12,34,2019-02-23 17:21:56,1,1,0


In [10]:
cols = ['age','area','behavior','connectionType','consuptionAbility','device','education','gender','status','work']
cols1 = ['state', 'price', 'time', 'people']

In [11]:
new_ads = ads
new_ads = new_ads.reindex(columns=new_ads.columns.to_list() + cols + cols1)

In [12]:
new_ads.head(7).T

,0,1,2,3,4,5,6
广告id,347637,347637,347637,347637,347637,347637,347637
创建时间,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-21 23:03:42
广告账户id,18529,18529,18529,18529,18529,18529,18529
商品id,28775,28775,28775,28775,28775,28775,28775
商品类型,13,13,13,13,13,13,13
广告行业id,12,12,12,12,12,12,12
素材尺寸,34,34,34,34,34,34,34
创建/修改时间,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-21 23:03:42,2019-02-22 23:37:02,2019-02-23 00:13:15,2019-02-23 05:37:13,2019-02-23 23:39:01
操作类型,2,2,2,1,1,1,1
修改字段,2,3,4,1,2,1,2


- 操作类型:1-修改,2-新建
- 修改字段:1-广告状态,2-出价,3-人群定向,4-广告时段设置

In [13]:
ops_field = {1: 'state', 2: 'price', 3: 'people', 4: 'time'}

In [14]:
new_ads.groupby(by=['操作类型', '修改字段']).min()

广告id                 创建时间  广告账户id  商品id  商品类型  广告行业id  素材尺寸  \
操作类型 修改字段                                                                
1    1       31  1970-01-01 00:00:00       6    -1     1      -1     1   
     2       32  1970-01-01 00:00:00      16    -1     1      -1     1   
     3      764  1970-01-01 00:00:00      80    -1     1       1     1   
     4      235  1970-01-01 00:00:00      34    -1     1      -1     1   
2    2       31  1970-01-01 00:00:00       6    -1     1      -1     1   
     3       31  1970-01-01 00:00:00       6    -1     1      -1     1   
     4       31  1970-01-01 00:00:00       6    -1     1      -1     1   

                       创建/修改时间  \
操作类型 修改字段                        
1    1     2017-06-06 01:44:47   
     2     2019-02-17 02:14:17   
     3     2019-02-17 02:14:17   
     4     2019-02-17 02:14:28   
2    2     1970-01-01 00:00:00   
     3     1970-01-01 00:00:00   
     4     1970-01-01 00:00:00   

                                                     操作后的字段值  age  ...  \
操作类型 修改字段                                                          ...   
1    1                                                     0  NaN  ...   
     2                                                     1  NaN  ...   
     3     age:1,217,731,739,333,393,988,366,608,942,638|...  NaN  ...   
     4                                         0,0,0,0,0,0,0  NaN  ...   
2    2                                                    10  NaN  ...   
     3     age:1,217,731,739,333,393,988,366,608,942,638|...  NaN  ...   
     4                                0,0,0,0,0,0,1073741823  NaN  ...   

           consuptionAbility  device  education  gender  status  work  state  \
操作类型 修改字段                                                                      
1    1                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     2                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     3                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     4                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
2    2                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     3                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     4                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   

           price  time  people  
操作类型 修改字段                       
1    1       NaN   NaN     NaN  
     2       NaN   NaN     NaN  
     3       NaN   NaN     NaN  
     4       NaN   NaN     NaN  
2    2       NaN   NaN     NaN  
     3       NaN   NaN     NaN  
     4       NaN   NaN     NaN  

[7 rows x 23 columns]

In [18]:
new_ads.groupby(by=['操作类型']).count()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,修改字段,操作后的字段值,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
操作类型,,,,,,,,,,,,,,,,,,,,,
1,527434,527434,527434,527434,527434,527434,527434,527434,527434,527434,...,0,0,0,0,0,0,0,0,0,0
2,86748,86748,86748,86748,86748,86748,86748,86748,86748,86748,...,0,0,0,0,0,0,0,0,0,0


- 按照操作类型排序，降表格分离开
- 527434 是修改 86748 是新建

In [32]:
new_ads.sort_values(by='操作类型', inplace=True)
new_ads.reset_index(drop=True, inplace=True)

In [33]:
new_ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
0,328480,2018-12-18 20:31:14,9953,25747,13,146,30,2019-03-09 22:56:49,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51068,2017-03-11 08:03:08,3953,-1,1,233,34,2019-03-02 15:11:18,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51068,2017-03-11 08:03:08,3953,-1,1,233,34,2019-03-16 22:34:25,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,238743,2019-01-11 16:06:05,27867,-1,1,171,1,2019-03-09 00:46:17,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,682808,2019-03-10 09:16:36,7230,-1,1,17,64,2019-03-11 09:38:58,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
new_ads.loc[527430:527438, :]

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527430,466304,2019-02-23 21:35:12,17333,17213,13,84,34,2019-03-10 05:35:01,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527431,667840,2018-12-19 20:37:25,13223,9965,13,17,30,2019-03-16 00:00:00,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527432,337216,2019-02-12 21:34:24,18284,3664,13,186,30,2019-03-03 10:09:54,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527433,466304,2019-02-23 21:35:12,17333,17213,13,84,34,2019-03-13 05:34:42,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527434,149229,2019-03-12 10:52:13,9087,12529,13,122,40,2019-03-12 10:52:13,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527435,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527436,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527437,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527438,227994,2019-03-04 07:51:59,14559,9601,13,122,30,2019-03-04 07:51:59,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
new_ads_fix = new_ads.loc[:527433, :]  # 包括边界
new_ads_crt = new_ads.loc[527434:, :]

In [40]:
new_ads_fix.tail()
new_ads_crt.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527429,394992,2019-02-21 23:03:29,18385,17284,13,146,34,2019-02-23 20:31:03,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527430,466304,2019-02-23 21:35:12,17333,17213,13,84,34,2019-03-10 05:35:01,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527431,667840,2018-12-19 20:37:25,13223,9965,13,17,30,2019-03-16 00:00:00,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527432,337216,2019-02-12 21:34:24,18284,3664,13,186,30,2019-03-03 10:09:54,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527433,466304,2019-02-23 21:35:12,17333,17213,13,84,34,2019-03-13 05:34:42,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527434,149229,2019-03-12 10:52:13,9087,12529,13,122,40,2019-03-12 10:52:13,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527435,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527436,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527437,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527438,227994,2019-03-04 07:51:59,14559,9601,13,122,30,2019-03-04 07:51:59,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 修改操作

In [42]:
new_ads_fix.fillna(value=-999, inplace=True)
new_ads_crt.fillna(value=-999, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
new_ads_fix.tail()
new_ads_crt.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527429,394992,2019-02-21 23:03:29,18385,17284,13,146,34,2019-02-23 20:31:03,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527430,466304,2019-02-23 21:35:12,17333,17213,13,84,34,2019-03-10 05:35:01,1,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527431,667840,2018-12-19 20:37:25,13223,9965,13,17,30,2019-03-16 00:00:00,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527432,337216,2019-02-12 21:34:24,18284,3664,13,186,30,2019-03-03 10:09:54,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527433,466304,2019-02-23 21:35:12,17333,17213,13,84,34,2019-03-13 05:34:42,1,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527434,149229,2019-03-12 10:52:13,9087,12529,13,122,40,2019-03-12 10:52:13,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527435,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527436,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,3,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527437,593247,2019-01-24 05:51:07,7077,-1,1,150,32,2019-01-24 05:51:07,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527438,227994,2019-03-04 07:51:59,14559,9601,13,122,30,2019-03-04 07:51:59,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [44]:
ops_field

{1: 'state', 2: 'price', 3: 'people', 4: 'time'}

In [50]:
new_ads_fix.groupby(by='广告id').count().head(10)

,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
广告id,,,,,,,,,,,,,,,,,,,,,
31,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
32,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
84,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
85,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
128,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
139,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
168,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
180,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
193,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [51]:
new_ads_fix.sort_values(by='创建时间').head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
364039,473241,1970-01-01 00:00:00,23975,17652,18,117,40,2019-03-14 11:16:54,1,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
166630,375352,1970-01-01 00:00:00,15038,-1,3,54,14,2019-02-28 23:46:22,1,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
51982,694446,1970-01-01 00:00:00,10118,-1,1,170,64,2019-03-14 15:46:19,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
51981,694446,1970-01-01 00:00:00,10118,-1,1,170,64,2019-03-14 15:43:10,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
51980,694446,1970-01-01 00:00:00,10118,-1,1,170,64,2019-03-14 15:41:29,1,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [52]:
ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,2,100
1,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,3,"age:217,601,79,202,837,942,638,394,347,731,739..."
2,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,4,"281474976694272,281474976694272,28147497669427..."
3,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-22 23:37:02,1,1,0
4,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-23 00:13:15,1,2,100
